# Use Your Data Wrangler Flow to Get Real-Time Predictions

Use Amazon SageMaker for real-time inference requests. The following notebook uses the Amazon SageMaker 
Python SDK to create an inference pipeline.

The notebook does the following:
1. Creates a SageMaker Pipeline.
2. Runs the pipeline.
    a. The pipeline uses the loan-default-inf-pipeline-DO-NOT-DEL.flow Data Wrangler flow file to prepare the data. All of the transformations that you’re exporting from your flow file are applied to the entire dataset.
    b. The pipeline trains either an Amazon SageMaker Autopilot model or an XGBoost model.
3. Creates a SageMaker inference pipeline.
4. Deploys the inference pipeline to an endpoint.
5. Uses the endpoint to make real-time predictions.
    a. The inference pipeline uses the Data Wrangler flow to transform the data from your inference request into a format that the trained model can use.
    b. The transformed data is provided to the trained model. The model uses the data to generate real-time predictions.

You can use your own predictive model in the notebook. To use your own model, skip the data preparation 
and model training steps. Navigate to the `Create SageMaker Inference Pipeline` section and specify the Amazon S3 location of the model.

For more information about inference pipelines, see [Host models along with pre-processing logic as serial inference pipeline behind one endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html).

## <font color='red'>Important</font>

<font color='red'>The transforms that you’ve applied to the target column are ignored at the time of inference. The 
target column is the column that your model is predicting. If your Data Wrangler flow contains a 
multicolumn transform that applies to the target column and other columns in your dataset, the 
transform is ignored for all the columns. The pipeline might therefore fail to perform inference.</font>

<font color='red'>If your flow has multicolumn transformations that transform the target, modify your Data Wrangler 
flow to have the transformation to the target column as its own transformation.</font>

# Install SageMaker Autopilot Dependencies
To use SageMaker Autopilot in your pipeline, you must install the newest versions of botocore, boto3, and sagemaker.

In [ ]:
!pip install -U 'botocore==1.*'
!pip install -U 'boto3==1.*'
!pip install -U 'sagemaker==2.*'

# Inputs and Outputs

The below settings configure the inputs and outputs for the flow export.

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

In <b>Input - Source</b> you can configure the data sources that will be used as input by Data Wrangler

1. For S3 sources, configure the source attribute that points to the input S3 prefixes
2. For all other sources, configure attributes like query_string, database in the source's 
<b>DatasetDefinition</b> object.

If you modify the inputs the provided data must have the same schema and format as the data used in the Flow. 
You should also re-execute the cells in this section if you have modified the settings in any data sources.

Parametrized data sources will be ignored when creating ProcessingInputs, and will directly read from the source.
Network isolation is not supported for parametrized data sources.
</div>

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition

data_sources = []

## Input - S3 Source: loans.csv

In [ ]:
data_sources.append(ProcessingInput(
    source="s3://sagemaker-us-west-2-251356100815/loan-default/loans.csv", # You can override this to point to other dataset on S3
    destination="/opt/ml/processing/loans.csv",
    input_name="loans.csv",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
))

## Output: S3 settings

<div class="alert alert-info"> 💡 <strong> Configurable Settings </strong>

1. <b>bucket</b>: you can configure the S3 bucket where Data Wrangler will save the output. The default bucket from 
the SageMaker notebook session is used. 
2. <b>flow_export_id</b>: A randomly generated export id. The export id must be unique to ensure the results do not 
conflict with other flow exports 
3. <b>s3_ouput_prefix</b>:  you can configure the directory name in your bucket where your data will be saved.
</div>

In [ ]:
import time
import uuid
import sagemaker

# Sagemaker session
sess = sagemaker.Session()

# You can configure this with your own bucket name, e.g.
# bucket = "my-bucket"
bucket = sess.default_bucket()
print(f"Data Wrangler export storage bucket: {bucket}")

# unique flow export ID
flow_export_id = f"{time.strftime('%d-%H-%M-%S', time.gmtime())}-{str(uuid.uuid4())[:8]}"
flow_export_name = f"flow-{flow_export_id}"

Below are the inputs required by the SageMaker Python SDK to launch a processing job.

In [ ]:
# Output name is auto-generated from the select node's ID + output name from the flow file.
output_name = "9a9ddd7b-f758-41ff-9b91-68a671a773bb.default"

s3_output_prefix = f"export-{flow_export_name}/output"
s3_output_base_path = f"s3://{bucket}/{s3_output_prefix}"
print(f"Processing output base path: {s3_output_base_path}\nThe final output location will contain additional subdirectories.")

processing_job_output = ProcessingOutput(
    output_name=output_name,
    source="/opt/ml/processing/output",
    destination=s3_output_base_path,
    s3_upload_mode="EndOfJob"
)

## Upload Flow to S3

To use the Data Wrangler as an input to the processing job,  first upload your flow file to Amazon S3.

In [ ]:
import os
import json
import boto3

# name of the flow file which should exist in the current notebook working directory
flow_file_name = "loan-default-inf-pipeline-DO-NOT-DEL.flow"

# Load .flow file from current notebook working directory 
!echo "Loading flow file from current notebook working directory: $PWD"

with open(flow_file_name) as f:
    flow = json.load(f)

# Upload flow to S3
s3_client = boto3.client("s3")
s3_client.upload_file(flow_file_name, bucket, f"data_wrangler_flows/{flow_export_name}.flow", ExtraArgs={"ServerSideEncryption": "aws:kms"})

flow_s3_uri = f"s3://{bucket}/data_wrangler_flows/{flow_export_name}.flow"

print(f"Data Wrangler flow {flow_file_name} uploaded to {flow_s3_uri}")

The Data Wrangler Flow is also provided to the Processing Job as an input source which we configure below.

In [ ]:
## Input - Flow: loan-default-inf-pipeline-DO-NOT-DEL.flow
flow_input = ProcessingInput(
    source=flow_s3_uri,
    destination="/opt/ml/processing/flow",
    input_name="flow",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type="FullyReplicated"
)

## Create Processor

In [ ]:
# IAM role for executing the processing job.
iam_role = sagemaker.get_execution_role()

# Unique processing job name. Give a unique name every time you re-execute processing jobs.
processing_job_name = f"data-wrangler-flow-processing-{flow_export_id}"

# Data Wrangler Container URL.
container_uri = "174368400705.dkr.ecr.us-west-2.amazonaws.com/sagemaker-data-wrangler-container:1.x"
# Pinned Data Wrangler Container URL. 
container_uri_pinned = "174368400705.dkr.ecr.us-west-2.amazonaws.com/sagemaker-data-wrangler-container:1.30.2"

# Processing Job Instance count and instance type.
instance_count = 2
instance_type = "ml.m5.4xlarge"

# Size in GB of the EBS volume to use for storing data during processing.
volume_size_in_gb = 30

# Content type for each output. Data Wrangler supports CSV as default and Parquet.
output_content_type = "CSV"

# Delimiter to use for the output if the output content type is CSV. Uncomment to set.
# delimiter = ","

# Compression to use for the output. Uncomment to set.
# compression = "gzip"

# Configuration for partitioning the output. Uncomment to set.
# "num_partition" sets the number of partitions/files written in the output.
# "partition_by" sets the column names to partition the output by.
# partition_config = {
#     "num_partitions": 1,
#     "partition_by": ["column_name_1", "column_name_2"],
# }

# Network Isolation mode; default is off.
enable_network_isolation = False

# List of tags to be passed to the processing job.
user_tags = []

# Output configuration used as processing job container arguments. Only applies when writing to S3.
# Uncomment to set additional configurations.
output_config = {
    output_name: {
        "content_type": output_content_type,
        # "delimiter": delimiter,
        # "compression": compression,
        # "partition_config": partition_config,
    }
}

# Refit configuration determines whether Data Wrangler refits the trainable parameters on the entire dataset. 
# When True, the processing job relearns the parameters and outputs a new flow file.
# You can specify the name of the output flow file under 'output_flow'.
# Note: There are length constraints on the container arguments (max 256 characters).
refit_trained_params = {
    "refit": False,
    "output_flow": f"data-wrangler-flow-processing-{flow_export_id}.flow"
}

# KMS key for per object encryption; default is None.
kms_key = None

### (Optional) Configure Spark Cluster Driver Memory

In [ ]:
# The Spark memory configuration. Change to specify the driver and executor memory in MB for the Spark cluster during processing.
driver_memory_in_mb = 2048
executor_memory_in_mb = 55742

config = json.dumps({
    "Classification": "spark-defaults",
    "Properties": {
        "spark.driver.memory": f"{driver_memory_in_mb}m",
        "spark.executor.memory": f"{executor_memory_in_mb}m"
    }
})

config_file = f"config-{flow_export_id}.json"
with open(config_file, "w") as f:
    f.write(config)

config_s3_path = f"spark_configuration/{processing_job_name}/configuration.json"
config_s3_uri = f"s3://{bucket}/{config_s3_path}"
s3_client.upload_file(config_file, bucket, config_s3_path, ExtraArgs={"ServerSideEncryption": "aws:kms"})
print(f"Spark Config file uploaded to {config_s3_uri}")
os.remove(config_file)

# Provides the spark config file to processing job and set the cluster driver memory. Uncomment to set.
# data_sources.append(ProcessingInput(
#     source=config_s3_uri,
#     destination="/opt/ml/processing/input/conf",
#     input_name="spark-config",
#     s3_data_type="S3Prefix",
#     s3_input_mode="File",
#     s3_data_distribution_type="FullyReplicated"
# ))

## Create Processer for the Pipeline

Create a Processor object. The pipeline uses the processor to apply the transformations from your flow file to the dataset.


In [ ]:
# Setup processing job network configuration
from sagemaker.network import NetworkConfig

network_config = NetworkConfig(
        enable_network_isolation=enable_network_isolation,
        security_group_ids=None,
        subnets=None
    )

In [ ]:
from sagemaker.processing import Processor

processor = Processor(
    role=iam_role,
    image_uri=container_uri,
    instance_count=instance_count,
    instance_type=instance_type,
    volume_size_in_gb=volume_size_in_gb,
    network_config=network_config,
    sagemaker_session=sess,
    output_kms_key=kms_key,
    tags=user_tags
)

# Create SageMaker Pipeline
A SageMaker pipeline is composed of a series of steps. You begin by creating a processing step to 
transform your data. If you’re using the notebook to train a model, you also define a training step. 

## Define Pipeline Steps
To create a SageMaker pipeline, create a `ProcessingStep` using the Data Wrangler processor defined 
in the preceding section.

In [ ]:
from sagemaker.workflow.steps import ProcessingStep

data_wrangler_step = ProcessingStep(
    name="DataWranglerProcessingStep",
    processor=processor,
    inputs=[flow_input] + data_sources, 
    outputs=[processing_job_output],
    job_arguments=[f"--output-config '{json.dumps(output_config)}'"] 
        + [f"--refit-trained-params '{json.dumps(refit_trained_params)}'"],
)

You can add a `TrainingStep` to the pipeline. The step trains a model on the transformed dataset. You 
can choose either an AutoML training step from SageMaker Autopilot or an XGBoost training step. By 
default, the AutoML training step is selected.


In [ ]:
add_training_step = True

To use the AutoML training step, run the following cells as-is. To use the XGBoost training step, set `use_automl_step` to False.

Currently, the AutoML training step only supports the `ENSEMBLING` mode. For more information about the AutoML training step, 
see [Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-automl).

In [ ]:
use_automl_step = True

For AutoML, specifying the target column of your dataset is required. 
The target column is the column that the model is trained to predict. 
Provide the name of the target column in the following cell.

In [ ]:
target_attribute_name = ""  # Provide the target column name here

if use_automl_step and not target_attribute_name:
    raise RuntimeError("You must specify the target column name.")

In [ ]:
if add_training_step and use_automl_step:
    from sagemaker import AutoML, AutoMLInput
    from sagemaker.workflow.automl_step import AutoMLStep
    from sagemaker.workflow.functions import Join
    from sagemaker.workflow.pipeline_context import PipelineSession

    pipeline_session = PipelineSession()

    training_input_content_type = None

    if output_content_type == "CSV":
        training_input_content_type = 'text/csv;header=present'
    elif output_content_type == "Parquet":
        training_input_content_type = 'x-application/vnd.amazon+parquet'

    auto_ml = AutoML(
        role=iam_role,
        target_attribute_name=target_attribute_name,
        sagemaker_session=pipeline_session,
        mode="ENSEMBLING"
    )

    s3_input = Join(
        on="/",
        values=[
            data_wrangler_step.properties.ProcessingOutputConfig.Outputs[output_name].S3Output.S3Uri,
            data_wrangler_step.properties.ProcessingJobName,
            f'{output_name.replace(".", "/")}',
        ]
    )

    train_args = auto_ml.fit(
        inputs=AutoMLInput(
            inputs=s3_input,
            content_type=training_input_content_type,
            target_attribute_name=target_attribute_name
        )
    )

    training_step = AutoMLStep(
        name="DataWrangerAutoML",
        step_args=train_args,
    )

The XGBoost algorithm uses the first column as the target column. If this is not the case for 
the data that you’ve processed, use the "Move column" transform to make the target column the 
first column.

In [ ]:
if add_training_step and not use_automl_step:
    import boto3
    from sagemaker.estimator import Estimator
    from sagemaker.workflow.functions import Join

    region = boto3.Session().region_name

    image_uri = sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=region,
        version="1.5-1",
        py_version="py3",
        instance_type=instance_type,
    )
    xgb_train = Estimator(
        image_uri=image_uri,
        instance_type=instance_type,
        instance_count=1,
        role=iam_role,
    )
    xgb_train.set_hyperparameters(
        objective="reg:squarederror",
        num_round=3,
    )

    from sagemaker.inputs import TrainingInput
    from sagemaker.workflow.steps import TrainingStep

    xgb_input_content_type = None

    if output_content_type == "CSV":
        xgb_input_content_type = 'text/csv'
    elif output_content_type == "Parquet":
        xgb_input_content_type = 'application/x-parquet'

    training_step = TrainingStep(
        name="DataWrangerTrain",
        estimator=xgb_train,
        inputs={
            "train": TrainingInput(
                s3_data=Join(
                    on="/",
                    values=[
                        data_wrangler_step.properties.ProcessingOutputConfig.Outputs[output_name].S3Output.S3Uri,
                        data_wrangler_step.properties.ProcessingJobName,
                        f'{output_name.replace(".", "/")}',
                    ]
                ),
                content_type=xgb_input_content_type
            )
        }
    )

## Define a Pipeline of Parameters, Steps
Now you will create the SageMaker pipeline that combines the steps created above so it can be executed. 

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The parameters supported in this notebook includes:

- `instance_type` - The ml.* instance type of the processing job.
- `instance_count` - The instance count of the processing job.

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
# Define Pipeline Parameters
instance_type = ParameterString(name="InstanceType", default_value="ml.m5.4xlarge")
instance_count = ParameterInteger(name="InstanceCount", default_value=1)

You will create a pipeline with the steps and parameters defined above

In [ ]:
import time
import uuid

from sagemaker.workflow.pipeline import Pipeline

# Create a unique pipeline name with flow export name
pipeline_name = f"pipeline-{flow_export_name}"

# Combine pipeline steps
pipeline_steps = [data_wrangler_step]
if add_training_step:
    pipeline_steps.append(training_step)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[instance_type, instance_count],
    steps=pipeline_steps,
    sagemaker_session=sess
)

### (Optional) Examining the pipeline definition

The JSON of the pipeline definition can be examined to confirm the pipeline is well-defined and 
the parameters and step properties resolve correctly.

In [ ]:
import json

definition = json.loads(pipeline.definition())
definition

## Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the SageMaker Pipeline service and start an execution. The role passed in 
will be used by the Pipeline service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=iam_role)
execution = pipeline.start()

## Pipeline Operations: Examine and Wait for Pipeline Execution

Describe the pipeline execution and wait for its completion.

In [ ]:
try:
    # Waiter will wait for up to 1 hour; increase the delay and max_attempts if necessary
    execution.wait(delay=30, max_attempts=120)
except Exception as e:
    listed_steps = execution.list_steps()
    errors = []
    for step in listed_steps:
        if "FailureReason" in step:
            errors.append(step["FailureReason"])
    raise RuntimeError(str(errors)) from e

List the steps in the execution. These are the steps in the pipeline that have been resolved by the step 
executor service.

In [ ]:
execution.list_steps()

You can visualize the pipeline execution status and details in Studio. For details please refer to 
[View, Track, and Execute SageMaker Pipelines in SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-studio.html)

## Data Wrangler Step S3 Output Location
The output of your Data Wrangler processing step will be printed below. To prevent data of different processing jobs 
and different output nodes from being overwritten or combined, Data Wrangler uses the name of the processing job and 
the name of the output to write the output.

In [ ]:
from sagemaker.processing import ProcessingJob

data_wrangler_job_arn = execution.list_steps()[-1]["Metadata"]["ProcessingJob"]["Arn"]
data_wrangler_job = ProcessingJob.from_processing_arn(sess, data_wrangler_job_arn)
data_wrangler_job_name = data_wrangler_job.describe()["ProcessingJobName"]

s3_job_results_path = f"{s3_output_base_path}/{data_wrangler_job_name}/{output_name.replace('.', '/')}"
print(f"Job results are saved to S3 path: {s3_job_results_path}")

# Create SageMaker Inference Pipeline

## Set Target Column
If the dataset that you used to create the Data Wrangler flow has a target column, you must specify 
it as the value of `target_column_name`. Set `contains_target_column` to False if you're not using 
a target column.

In [ ]:
byo_model = False

In [ ]:
contains_target_column = True

In [ ]:
if byo_model or not use_automl_step:
    target_column_name = ""  # Provide the target column name here
else:
    target_column_name = target_attribute_name

if contains_target_column and not target_column_name:
    raise RuntimeError("You must specify the target column name.")

## Define Model Locations
Data Wrangler has created a modified version of your flow file to make it usable for inference. 
The following notebook cells upload the modified Data Wrangler flow to Amazon S3 and get the S3 
location of the model that you’ve trained. If you’re using your own model, specify its S3 
location as the value of algo_model_uri and  set byo_model to True.

In [ ]:
if byo_model:
    algo_model_uri = ""  # Provide your model S3 URI here
elif use_automl_step:
    automl_job_arn = execution.list_steps()[0]["Metadata"]["AutoMLJob"]["Arn"]
    automl_job_name = automl_job_arn.split("/")[1]
    automl_job_desc = sess.describe_auto_ml_job(automl_job_name)
    best_inference_container = automl_job_desc["BestCandidate"]["InferenceContainers"][0]
    algo_model_uri = best_inference_container["ModelDataUrl"]
else:
    training_job_arn = execution.list_steps()[0]["Metadata"]["TrainingJob"]["Arn"]
    training_job_name = training_job_arn.split("/")[1]
    training_job_desc = sess.describe_training_job(training_job_name)
    algo_model_uri = training_job_desc["ModelArtifacts"]["S3ModelArtifacts"]

print(f"Model URI: {algo_model_uri}")

In [ ]:
import sagemaker

# Sagemaker session
sess = sagemaker.Session()

# You can configure this with your own bucket name, e.g.
# inference_bucket = "my-bucket"
inference_bucket = sess.default_bucket()

print(f"Data Wrangler export storage bucket: {inference_bucket}")

In [ ]:
import os
import boto3

# name of the inference flow model which should exist in the current notebook working directory
inference_flow_name = "loan-default-inf-pipeline-DO-NOT-DEL_2022-11-19-01-36-10.tar.gz"
inference_flow_path = f"/home/sagemaker-user/data/data_wrangler_inference_flows/{inference_flow_name}"

# Upload inference flow model to S3
s3_client = boto3.client("s3")
s3_client.upload_file(inference_flow_path, inference_bucket, f"data_wrangler_inference_flows/{inference_flow_name}", ExtraArgs={"ServerSideEncryption": "aws:kms"})

inference_flow_uri = f"s3://{inference_bucket}/data_wrangler_inference_flows/{inference_flow_name}"

print(f"Data Wrangler inference flow model URI: {inference_flow_uri}")

## Create SageMaker Models
The following cells use the model locations that you’ve specified to create models in SageMaker. 
You can also include additional models in the inference pipeline. To include additional models, 
define them in the following cells and append them to `pipeline_models`.

In [ ]:
from datetime import datetime

timestamp = datetime.strftime(datetime.now(), '%Y-%m-%d-%H-%M-%S')

# IAM role for the models.
iam_role = sagemaker.get_execution_role()

dw_container_uri = "174368400705.dkr.ecr.us-west-2.amazonaws.com/sagemaker-data-wrangler-container:1.x"

if byo_model:
    algo_container_uri = ""  # Provide the ECR image of the algorithm used to train your model
elif use_automl_step:
    algo_container_uri = best_inference_container["Image"]
else:
    algo_container_uri = image_uri

In [ ]:
pipeline_models = []

The following cell defines and appends the data_wrangler_model. The Data Wrangler model is 
the data processing step in the inference pipeline that you’re creating. It uses the 
transformations that you’ve made in the Data Wrangler flow.

In [ ]:
from sagemaker.model import Model

data_wrangler_model_name = f"DataWranglerInferencePipelineFlowModel-{timestamp}"

data_wrangler_model = Model(
    image_uri=dw_container_uri,
    model_data=inference_flow_uri,
    role=iam_role,
    name=data_wrangler_model_name,
    sagemaker_session=sess,
    env={"INFERENCE_TARGET_COLUMN_NAME": target_column_name}
)

pipeline_models.append(data_wrangler_model)

The following cell defines and appends the algo_model.  The algorithm model is the predictive 
step in the inference pipeline. It’s either the model that you’ve trained in the notebook or 
a model that you’ve trained elsewhere.

In [ ]:
algo_model_name = f"DataWranglerInferencePipelineAlgoModel-{timestamp}"
algo_environment = None  # Provide your model environment variables here if bringing your own model

if not byo_model and use_automl_step:
    algo_environment = best_inference_container["Environment"]

algo_model = Model(
    image_uri=algo_container_uri,
    model_data=algo_model_uri,
    role=iam_role,
    name=algo_model_name,
    sagemaker_session=sess,
    env=algo_environment,
)

pipeline_models.append(algo_model)

## Create SageMaker Inference Pipeline
Use the models that you’ve defined to create a PipelineModel. The pipeline model passes the 
intermediate output of the Data Wrangler model to the algorithm model.

For models, the order of the list of models is the order of the pipeline. The pipeline won’t 
work if the Data Wrangler model isn’t first.

In [ ]:
from sagemaker.pipeline import PipelineModel

inference_pipeline_model_name = f"DataWranglerInferencePipelineModel-{timestamp}"

inference_pipeline_model = PipelineModel(
    models=pipeline_models,
    role=iam_role,
    name=inference_pipeline_model_name,
    sagemaker_session=sess,
)

The following cells deploy the pipeline model to an endpoint. You must deploy the pipeline 
model to an endpoint before you can make inference requests.

In [ ]:
# Adjust the endpoint configuration as needed
instance_type = "ml.m5.xlarge"
instance_count = 1

endpoint_name = f"DataWranglerInferencePipelineEndpoint-{timestamp}"

In [ ]:
inference_pipeline_model.deploy(instance_count, instance_type, endpoint_name=endpoint_name)

# Invoke the Inference Pipeline with a Predictor

## Create a Predictor
Create a predictor to make requests to your endpoint.

In [ ]:
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name,
    sagemaker_session=sess,
)

## Make a Sample Inference Request
The following cells use a data point from your dataset to get a prediction from the endpoint. 
The data point doesn’t have any transformations from your Data Wrangler flow applied to it.

In [ ]:
sample_record_csv = '''loan_status,loan_amount,funded_amount_by_investors,loan_term,interest_rate,installment,grade,sub_grade,verification_status,issued_on,purpose,dti,earliest_credit_line,inquiries_last_6_months,open_credit_lines,derogatory_public_records,revolving_line_utilization_rate,total_credit_lines,employer_title,home_ownership,employment_length,annual_income
fully paid,5000,4975.0,36,10.65,162.87,b,b2,verified,2011-12-01,credit_card,27.65,1985-01-01,1,3,0,83.7,9,,rent,10.0,24000.0
'''

In [ ]:
import pandas as pd
from io import StringIO

sample_record_io = StringIO(sample_record_csv)
sample_df = pd.read_csv(sample_record_io)

if contains_target_column:
    # Drop the target column from the sample csv
    print(f"Target column value of sample record: {sample_df.iloc[0][target_column_name]}")
    sample_df = sample_df.drop(columns=[target_column_name])

sample_record_payload = sample_df.to_csv(header=False, index=False)
print(f"Sample record to predict: {sample_record_payload}")

In [ ]:
prediction = predictor.predict(sample_record_payload, initial_args={"ContentType": "text/csv"})

print(f"The predicted target is: {prediction}")

## (Optional) Cleanup
To delete the SageMaker resources created in this notebook, set `resource_deletion` to True.


In [ ]:
resource_deletion = False

In [ ]:
if resource_deletion:
    pipeline.delete()

In [ ]:
if resource_deletion:
    predictor.delete_endpoint()
    inference_pipeline_model.delete_model()